# Load Data into a Database

* Install faker to fake PII data

In [1]:
%pip install faker
%pip install keybert
%pip install --upgrade torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  DEPRECATION: Configuring installation scheme with

In [2]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
from faker import Faker # To fake PII data like names
import random
from keybert import KeyBERT

In [3]:
fake = Faker()

## Connect to Database
* In reality the credentials would be stored in a secure way like with Secrets Mangager

In [4]:
DB_NAME = "legacy_pacheco"
DB_USER = "rpacheco"
DB_PASSWORD = "D1qoGvjXqPTd2vB4i4KS71xDEWmCOpSq"
DB_HOST = "dpg-d1ff63mmcj7s739p963g-a.oregon-postgres.render.com"
DB_PORT = "5432"

In [5]:
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT,
    sslmode="require"
)
cursor = conn.cursor()

## Read in Kaggle NLP Mental Health Conversations Dataset
* https://www.kaggle.com/datasets/thedevastator/nlp-mental-health-conversations?resource=download

In [6]:
df = pd.read_csv('train.csv')

In [7]:
df.head()

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


* Assign fake PII data. I chose first/last name, clinic (city), doctor, and age
* This is to train a model to generate a numerical/categorical response
* Use a KeyBERT model to identify keywords for each Context column. This is used in the database search feature in the app that pairs doctor responses to keywords.

In [8]:
kw_model = KeyBERT()

In [9]:
clinics = [fake.city() for x in range(0, 100)]

In [10]:
df["firstName"] = [fake.first_name() for _ in range(len(df))]
df["lastName"] = [fake.last_name() for _ in range(len(df))]
df["clinic"] = [clinics[int(random.randint(0,99))] for _ in range(len(df))]
df["doctor"] = [fake.last_name() for _ in range(len(df))]
df["age"] = [int(random.randint(18, 50)) for _ in range(len(df))] # keeping ages in the 18 - 50 range so data isn't to spread out
df["keywords"] = df["Context"].apply(lambda x: [kw[0] for kw in kw_model.extract_keywords(x)])

In [11]:
df.head()

,Context,Response,firstName,lastName,clinic,doctor,age,keywords
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb...",Shawn,Salas,Williamport,Kennedy,20,"[suicide, worthless, sleep, feeling, change]"
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see...",Emily,Thornton,Fergusonbury,Gonzalez,43,"[suicide, worthless, sleep, feeling, change]"
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...,Brandon,George,Kristopherport,Galvan,26,"[suicide, worthless, sleep, feeling, change]"
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...,Gary,Brown,Ryanberg,Vargas,29,"[suicide, worthless, sleep, feeling, change]"
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...,William,Howell,Markberg,Hawkins,38,"[suicide, worthless, sleep, feeling, change]"


* Convert dataframe to a numpy array to insert into postgres

In [12]:
data = df.to_records(index=False).tolist()

* Generate insert query and insert into table

In [13]:
data[0]

("I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad feeling

In [14]:
insert_query = """
    INSERT INTO records (context, response, firstName, lastName, clinic, doctor, age, keywords)
    VALUES %s
"""
execute_values(cursor, insert_query, data)

conn.commit()

In [15]:
cursor.close()
conn.close()